In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
model = EfficientPhys(TN=True)
model.compile(optimizer='adamw', loss='mse')
model(np.random.random((4, 160, 72, 72, 3)));
model.summary()

Model: "efficient_phys"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tsm (TSM)                       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (640, 72, 72, 32)      │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (640, 70, 70, 32)      │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (640, 35, 35, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (640, 33, 33, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (640, 70, 70, 32)      │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_mask (Attention_mask) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (640, 33, 33, 64)      │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (640, 35, 35, 32)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (640, 35, 35, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (640, 128)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (640, 128)             │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (640, 1)               │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (640, 5184, 3)         │        20,736 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (640, 16384)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,188,929 (8.35 MB)

 Trainable params: 2,178,561 (8.31 MB)

 Non-trainable params: 10,368 (40.50 KB)

In [3]:
batch_size = 32

rlap_tape = "/root/ssd_cache/rppg_training_data/rlap_160x72x72_all"

train_rlap = load_datatape(rlap_tape, fold='train', batch=batch_size)
valid_rlap = load_datatape(rlap_tape, fold='val', extended_hr='False', batch=batch_size)

train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_rlap), KerasDataset(valid_rlap)

training_set = training_set.apply_fn(compress_aug)

rlap_160x72x72_all             datatape has been loaded.    24704 items total. fold=train selected
rlap_160x72x72_all             datatape has been loaded.     3040 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/efficientphys.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/efficientphystn.weights.h5')
model.load_weights('../weights/efficientphystn.weights.h5')

Training:   0%|          | 0/772 [00:00<?, ?it/s]

HTML(value='')

Validating:   0%|          | 0/95 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.6256	Validation loss:0.5482	


Training:   0%|          | 0/772 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/95 [00:00<?, ?it/s]

Epoch  19:  Training loss: 0.473	Validation loss:0.4825	


Training:   0%|          | 0/772 [00:00<?, ?it/s]

Validating:   0%|          | 0/95 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.4731	Validation loss:0.4848	


In [5]:
eval_on_dataset(dataset_H5_mmpd, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhystn_RLAP_MMPD.h5')
get_metrics('../results/EfficientPhystn_RLAP_MMPD.h5', dropped='False')

  0%|          | 0/660 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,15.639±0.231,25.096±3.712,18.005±0.26,0.29833
Whole,10.418±0.486,16.23±4.701,11.645±0.499,0.50088


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhystn_RLAP_COHFACE.h5', fps=30)
get_metrics('../results/EfficientPhystn_RLAP_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,3.613±0.21,9.664±3.057,4.9±0.274,0.7073
Whole,2.822±0.436,6.251±3.557,3.8±0.539,0.8578


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhystn_RLAP_PURE.h5')
get_metrics('../results/EfficientPhystn_RLAP_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,0.622±0.057,1.688±0.905,0.92±0.089,0.99729
Whole,0.308±0.074,0.649±0.495,0.443±0.129,0.99961


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhystn_RLAP_UBFC.h5')
get_metrics('../results/EfficientPhystn_RLAP_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.38±0.097,2.602±1.166,1.417±0.102,0.9884
Whole,0.588±0.108,0.914±0.581,0.62±0.125,0.99872
